In [266]:
import idx2numpy
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import random as rnd
import numpy.typing as npt
import scipy.signal as sps
import string as str

IMAGE_EDGE_SIZE = 28
PIXELS_PER_IMAGE = IMAGE_EDGE_SIZE ** 2
CLASSES_COUNT = 10
ITERATIONS = 500
LEARNING_RATE = 0.15

In [267]:
def one_hot(labels: npt.ArrayLike) -> npt.NDArray:
    """
    Converts a 1D array of labels (the ground truth) to 2D matrix of shape (10, labels.size) as a probability distribution, 
    where the corresponding row given by the label value has probability of 1.
    
    :labels: The ground truth.
    :return: Encoded values of labels to probability distribution.
    """
    one_hot = np.zeros((10, labels.size))
    one_hot[labels, np.arange(labels.size)] = 1
    return one_hot

def get_accuracy(results: npt.NDArray, labels: npt.ArrayLike) -> float:
    """
    Calculates the accuracy of a neural network from the results of classification by comparing it to the ground truth.

    :results: The forward propagation results.
    :labels: The ground truth.
    :return: The accuracy as a real number. 
    """
    return (np.sum(np.argmax(results, 0) == labels) / labels.size)

def show_some_mistakes(results: npt.NDArray, labels: npt.ArrayLike, data: npt.NDArray, samples = 10) -> None:
    """
    Plots randomly choosen images, which were not classified correctly.

    :results: The forward propagation results.
    :labels: The ground truth.
    :data: The input data of forward propagation, i.e images.
    :samples: The number of shown images, 10 by default.
    """
    results = np.argmax(results, 0)
    i = rnd.randint(0, labels.size)
    j = 0
    while j < samples:
        i = (i + 1) % labels.size
        if results[i] != labels[i]:
            print("labeled:", labels[i], "-- classified:", results[i])
            plt.imshow(data[:, i].reshape((IMAGE_EDGE_SIZE, IMAGE_EDGE_SIZE)), cmap='gray')
            plt.show()
            j += 1

def random_name():
    return "".join(rnd.choices(str.ascii_letters + str.digits, k=8))

In [268]:
def ReLU(L: npt.NDArray) -> npt.NDArray:
    """
    Calculates the Rectified Linear Units of a numpy matrix.
    
    :L: Matrix of values of a hidden layer.
    :return: For all nonnegative numbers returns its value, otherwise 0.
    """
    return np.maximum(0, L)

def ReLU_deriv(L: npt.NDArray) -> npt.NDArray:
    """
    Calculates the derivation of ReLu function of a numpy matrix.

    :L: Matrix of values of a hidden layer.
    :return: For all positive numbers returns 1, otherwise 0.
    """
    return L > 0

def sigmoid(L: npt.NDArray) -> npt.NDArray:
    """
    Calculates the Sigmoid function of a numpy matrix.
    
    :L: Values of a hidden layer.
    :return: For all indexes with value x returns 1 / (1 + e^(-x)).
    """
    return 1 / (1 + np.exp(-L))

def softmax(L: npt.NDArray) -> npt.NDArray:
    """
    Converts matrix of N values in a row to probability distribution of N outcomes for each row.

    :L: Values of an output layer.
    :return: For all indexes of the given matrix returns the probability of a given index in its row.
    """
    return np.exp(L) / sum(np.exp(L))

In [269]:
def load_training_data() -> tuple:
    """
    Loads training data and training labels from files and transforms them to desired shape.

    :return: Matrix of training data and array of training labels.
    """
    training_data = idx2numpy.convert_from_file("mnist/train-images.idx3-ubyte") / 255
    training_labels = idx2numpy.convert_from_file("mnist/train-labels.idx1-ubyte")
    return training_data, training_labels

def load_test_data() -> tuple:
    """
    Loads testing data and training labels from files and transforms them to desired shape.

    :return: Matrix of testing data and array of testing labels.
    """
    test_data = idx2numpy.convert_from_file("mnist/t10k-images.idx3-ubyte") / 255
    test_labels = idx2numpy.convert_from_file("mnist/t10k-labels.idx1-ubyte")
    return test_data, test_labels

In [270]:
class Layer():
    def __init__(self, name):
        self.name = name
        
    def forward():
        pass

    def adjust():
        pass

    def backward():
        pass

    def save(self):
        try:
            np.save(self.name + "_K.npy", self.kernels)
        except:
            pass
    
        try:
            np.save(self.name + "_W.npy", self.weights)
        except:
            pass
    
        try:
            np.save(self.name + "_B.npy", self.biases)
        except:
            pass

    def load(self, kernels: bool, weights: bool, biases: bool):
        if kernels:
            try:
                self.kernels = np.load(self.name + "_K.npy")
            except:
                kernels = False
        
        if weights:
            try:
                self.weights = np.load(self.name + "_W.npy")
            except:
                weights = False
        
        if biases:
            try:
                self.biases = np.load(self.name + "_B.npy")
            except:
                biases = False
        
        return kernels, weights, biases

In [271]:
class ConvolutionLayer(Layer):
    def __init__(self, kernel_count, kernel_size, activation, name = random_name()):
        Layer.__init__(self, name)
        self.activation = activation
        kernels, _, biases = self.load(True, False, True)
        if not kernels:
            self.kernels = np.random.rand(kernel_count, kernel_size, kernel_size) - 0.5
        if not biases:
            self.biases = np.random.rand(kernel_count) - 0.5

    def forward(self, input):
        self.input = input
        kernel_count = self.kernels.shape[0]
        input_count = input.shape[0]
        output = np.zeros((input_count * kernel_count, input.shape[1] - 2, input.shape[2] - 2))
        
        for i in range(input_count):
            k = i * kernel_count
            for j in range(kernel_count):
                output[k + j] = self.activation(sps.correlate2d(input[i], self.kernels[j], "valid") + self.biases[j])
        
        return output

In [272]:
class MaxPoolLayer(Layer):
    def __init__(self, size, name = random_name()):
        Layer.__init__(self, name)
        self.size = size
        
    def forward(self, input):
        input_count = input.shape[0]
        pooled_width = int(input.shape[1] / self.size)
        pooled_height = int(input.shape[2] / self.size)
        pooled = np.zeros((input_count, pooled_width, pooled_height))
        
        for i in range(input_count):
            for j in range(pooled_width):
                for k in range(pooled_height):
                    l = j * self.size
                    m = k * self.size
                    pooled[i, j, k] = np.max(input[i, l:l+self.size, m:m+self.size])
        
        return pooled
    
    def adjust():
        #np.argmax(window, axis=0, keepdims=True,)[0]
        pass

In [273]:
class FlatteningLayer(Layer):
    def __init__(self, sample_count, channel_count, name = random_name()):
        Layer.__init__(self, name)
        self.sample_count = sample_count
        self.channel_count = channel_count

    def forward(self, input):
        return np.reshape(input, (self.sample_count, input.shape[1] * input.shape[2] * self.channel_count)).T

In [274]:
class FullyConnectedLayer(Layer):
    def __init__(self, input_size, output_size, activation, activation_deriv, name = random_name()):
        Layer.__init__(self, name)
        self.activation = activation
        self.activation_deriv = activation_deriv
        _, weights, biases = self.load(False, True, True)
        if not weights:
            self.weights = np.random.rand(output_size, input_size) - 0.5
        if not biases:
            self.biases = np.random.rand(output_size, 1) - 0.5
    
    def forward(self, input):
        self.input = input
        return self.activation(self.weights.dot(input) + self.biases)
    
    def adjust(self, dOutput, sample_count, learning_rate):
        self.weights = self.weights - (dOutput.dot(self.input.T) / sample_count) * learning_rate
        self.biases = self.biases - (np.sum(dOutput) / sample_count) * learning_rate
    
    def backward(self, dOutput):
        return self.weights.T.dot(dOutput) * self.activation_deriv(self.input)



In [276]:
class NeuralNetwork():
    def __init__(self, training_data, training_labels, *layers: Layer):
        self.training_data = training_data
        self.sample_count = training_data.shape[0]
        self.training_labels = training_labels
        self.one_hot_training_labels = one_hot(training_labels)
        self.layers = layers
    
    def train(self, iterations, learning_rate):
        for _ in range(iterations): 
            input = self.training_data
            for layer in self.layers:
                input = layer.forward(input)

            dOutput = input - self.one_hot_training_labels
            for i in range(len(self.layers) - 1, 0, -1):
                self.layers[i].adjust(dOutput, self.sample_count, learning_rate)
                dOutput = self.layers[i].backward(dOutput)

            self.layers[0].adjust(dOutput, self.sample_count, learning_rate)
        
    def save(self):
        for layer in self.layers:
            layer.save()

    def assess(self, test_data, test_labels, on_trainig_set = True):
        input = test_data
        for layer in self.layers:
            input = layer.forward(input)

        accuracy = (np.sum(np.argmax(input, 0) == test_labels) / test_labels.size)
        print("\n############################# Neural Network Results #############################\n")
        print("Accuracy on test set: ", accuracy)

        if on_trainig_set:
            input = self.training_data
            for layer in self.layers:
                input = layer.forward(input)
            
            accuracy = (np.sum(np.argmax(input, 0) == self.training_labels) / self.training_labels.size)
            print("Accuracy on training set: ", accuracy)

            



In [277]:
training_data, training_labels = load_training_data()
sample_count = training_data.shape[0]
training_data = np.reshape(training_data, (sample_count, -1)).T

neural_network = NeuralNetwork(training_data, training_labels, FullyConnectedLayer(784, 56, ReLU, None, "FCL1"), FullyConnectedLayer(56, 10, softmax, ReLU_deriv, "OL"))
neural_network.train(50, 0.15)
neural_network.assess(training_data, training_labels, False)

#conv_layer1 = ConvolutionLayer(2, 3, ReLU)
#conv_layer2 = ConvolutionLayer(2, 3, ReLU)
#max_pool_layer = MaxPoolLayer(2)
#flat_layer = FlatteningLayer(2, 4)
#
#output = conv_layer1.forward(training_data[0:2])
#output = conv_layer2.forward(output)
#output = max_pool_layer.forward(output)
#flat = flat_layer.forward(output)
#
#for i in output:
#    plt.imshow(i, cmap='gray')
#    plt.show()
#
#print(flat.shape)
#plt.imshow(np.reshape(flat[0:144, 0], (12, 12)), cmap='gray')
#plt.show()


/tmp/ipykernel_4102/2070122192.py:35: RuntimeWarning: overflow encountered in exp
  return np.exp(L) / sum(np.exp(L))
/tmp/ipykernel_4102/2070122192.py:35: RuntimeWarning: invalid value encountered in divide
  return np.exp(L) / sum(np.exp(L))



############################# Neural Network Results #############################

Accuracy on test set:  0.09871666666666666
